In [2]:
import csv
import numpy as np
import networkx as nx
import math
from scipy.stats import mode
import function
import pickle
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist  

# 用户活动中心

In [5]:
# 得到所有的点
i = 0
csv_reader = csv.reader(open('./data/train_hard.csv', encoding='utf-8'))
node_set = set()
for row in csv_reader:
    i+=1
    if i == 1:
        continue
    node_set.add(row[5])
    node_set.add(row[6])
# 得到这些点对呀的lat long
node_lat_long = {}
for node in node_set:
    decode = function.decode_exactly(node)
    node_lat_long[node] = [decode[0],decode[1]]

In [ ]:
# 得到用户使用的起点和终点，大于10公里的行程忽略

i = 0
csv_reader = csv.reader(open('./data/train_hard.csv', encoding='utf-8'))
user_use_node = {}
for row in csv_reader:
    i+=1
    if i == 1:
        print(row[9])
        continue
    
    user = row[1]
    start = row[5]
    end = row[6]
    man_dis = row[9]
    
    if int(man_dis) > 10000:# 大于10
        continue
    
    if user not in user_use_node:
        user_use_node[user] = []
        user_use_node[user].append(start)
        user_use_node[user].append(end)
    else:
        user_use_node[user].append(start)
        user_use_node[user].append(end)



In [ ]:
# 得到用户使用的点的[lat long]

user_use_long_lat = {}
for user in user_use_node:
    user_use_long_lat[user] = []
    for node in user_use_node[user]:
        user_use_long_lat[user].append(node_lat_long[node])

In [ ]:
# 得到用户骑行的起点和终点的lat long差, 大于10公里不算

i = 0
csv_reader = csv.reader(open('./data/train_hard.csv', encoding='utf-8'))
user_infor = {}
for row in csv_reader:
    i+=1
    if i == 1:
        continue
    
    user = row[1]
    start = row[5]
    end = row[6]
    man_dis = row[9]
    
    if int(man_dis) > 10000:# 大于10
        continue
    
    if user not in user_infor:
        user_infor[user] = [[],[]]
        user_infor[user][0].append(abs(node_lat_long[start][0] - node_lat_long[end][0])*1000)# 存下所有的lat差
        user_infor[user][1].append(abs(node_lat_long[start][1] - node_lat_long[end][1])*1000)# 存下所有的long差
    else:
        user_infor[user][0].append(abs(node_lat_long[start][0] - node_lat_long[end][0])*1000)
        user_infor[user][1].append(abs(node_lat_long[start][1] - node_lat_long[end][1])*1000)

In [ ]:
# 得到用户的起点和终点的lat long差和的平均值
for user in user_infor:
    manhattan_lat_long_diff = np.mean(user_infor[user][0])+np.mean(user_infor[user][1])
    user_infor[user] = manhattan_lat_long_diff
    

In [ ]:
#根据节点使用时间-次数占比聚簇

#用scipy求解距离

i = 0

user_center = {}
for user in user_use_long_lat:
    
    i+=1
    
    if i%10000 == 0:
        print(i)
    
    K=range(1,20)
    X = user_use_long_lat[user]
    for k in K:
        kmeans=KMeans(n_clusters=k)
        kmeans.fit(X)
        
       
        if sum(np.min(
                cdist(X,kmeans.cluster_centers_,
                     'cityblock'),axis=1))*1000/len(X) < user_infor[user]:
            break
        
    user_center[user] = []
    user_center[user].append(kmeans.cluster_centers_)
    
        
    




In [ ]:
record1 = 0
record2 = 0
record3 = 0
record4 = 0
record5 = 0
recordM = 0
for user,node in user_center.items():
    
    if len(node[0]) == 1:
        record1+=1
    elif len(node[0]) == 2:
        record2+=1
    elif len(node[0]) == 3:
        record3+=1
    elif len(node[0]) == 4:
        record4+=1
    elif len(node[0]) == 5:
        record5+=1
    else:
        recordM+=1

In [ ]:
print(record1)
print(record2)
print(record3)
print(record4)
print(record5)
print(recordM)

In [ ]:
with open('./data/temp/Incomplete_caching/user_center','wb+') as f:
    pickle.dump(user_center, f, pickle.HIGHEST_PROTOCOL)
    


# 单车活动中心

In [3]:
# 得到bike使用的起点和终点，大于10公里的行程忽略

i = 0
csv_reader = csv.reader(open('./data/train_hard.csv', encoding='utf-8'))
bike_use_node = {}
for row in csv_reader:
    i+=1
    if i == 1:
        print(row[9])
        continue
    
    bike = row[2]
    start = row[5]
    end = row[6]
    man_dis = row[9]
    
    if int(man_dis) > 10000:# 大于10
        continue
    
    if bike not in bike_use_node:
        bike_use_node[bike] = []
        
    bike_use_node[bike].append(start)
    bike_use_node[bike].append(end)
    



start_to_end_man_dis


In [6]:
# 得到bike使用的点的[lat long]

bike_use_long_lat = {}
for bike in bike_use_node:
    bike_use_long_lat[bike] = []
    for node in bike_use_node[bike]:
        bike_use_long_lat[bike].append(node_lat_long[node])

In [7]:
# 得到bike的起点和终点的lat long差, 大于10公里不算
i = 0
csv_reader = csv.reader(open('./data/train_hard.csv', encoding='utf-8'))
bike_infor = {}
for row in csv_reader:
    i+=1
    if i == 1:
        print(row[9])
        continue
    
    bike = row[2]
    start = row[5]
    end = row[6]
    man_dis = row[9]
    
    if int(man_dis) > 10000:# 大于10
        continue
    
    if bike not in bike_infor:
        bike_infor[bike] = [[],[]]
        
    bike_infor[bike][0].append(abs(node_lat_long[start][0] - node_lat_long[end][0])*1000)# 存下所有的lat差
    bike_infor[bike][1].append(abs(node_lat_long[start][1] - node_lat_long[end][1])*1000)# 存下所有的long差
    

start_to_end_man_dis


In [8]:
# 得到bike的起点和终点的lat long差和的平均值
for bike in bike_infor:
    manhattan_lat_long_diff = np.mean(bike_infor[bike][0])+np.mean(bike_infor[bike][1])
    bike_infor[bike] = manhattan_lat_long_diff
    

In [11]:
len(bike_use_long_lat)

463440

In [12]:
#根据节点使用时间-次数占比聚簇

#用scipy求解距离

i = 0

bike_center = {}
for bike in bike_use_long_lat:
    
    i+=1
    
    if i%10000 == 0:
        print(i)
    
    K=range(1,40)
    X = bike_use_long_lat[bike]
    for k in K:
        kmeans=KMeans(n_clusters=k)
        kmeans.fit(X)
        
       
        if sum(np.min(
                cdist(X,kmeans.cluster_centers_,
                     'cityblock'),axis=1))*1000/len(X) < bike_infor[bike]:
            break
        
    bike_center[bike] = []
    bike_center[bike].append(kmeans.cluster_centers_)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000


In [17]:
with open('./data/temp/Incomplete_caching/bike_center','wb+') as f:
    pickle.dump(bike_center, f, pickle.HIGHEST_PROTOCOL)
